In [ ]:
from gbmbkgpy.io.downloading import download_gbm_file, download_trigdata_file
from gbmgeometry import PositionInterpolator, GBM
from astropy.coordinates import SkyCoord
from gbmgeometry.utils.gbm_time import GBMTime
import astropy.time as time
import astropy.units as u
import pandas as pd
from datetime import datetime,timedelta
from morgoth.utils.trig_reader import TrigReader
from morgoth.auto_loc.time_selection import TimeSelectionBB
from astromodels.functions import Powerlaw, Cutoff_powerlaw, Band
from astromodels.sources.point_source import PointSource
from astromodels.functions.priors import Log_uniform_prior,Uniform_prior
from astromodels.core.model import Model
from threeML.data_list import DataList
from threeML.bayesian.bayesian_analysis import BayesianAnalysis,BayesianResults
from threeML.plugins.OGIPLike import OGIPLike
import os

In [3]:
lu = ["n0","n1","n2","n3","n4","n5","n6","n7","n8","n9","na","nb","b0","b1"]

Specify GRB and Download Data

In [4]:
GRB = "GRB230826814"
day_seconds = 24*60*60
grb_date = datetime(int(f"20{GRB.strip('GRB')[:2]}"),int(GRB.strip("GRB")[2:4]),int(GRB.strip("GRB")[4:6]))
grb_time = grb_date + timedelta(seconds=int(GRB[-3:])*day_seconds/1000)

# get swift grb table and look for coinciding
swift_table = pd.read_csv("swift_grbs.txt", sep = "\t", decimal = ".", encoding='latin-1', index_col=False)
swift_table.insert(1, "Date", [i[0:-1] for i in swift_table["GRB"]], True)
coinc = swift_table.loc[swift_table["Date"] == GRB.strip("GRB")[:-3]]

print(f"Total number of {len(coinc['Date'])} Swift trigger(s) found")

swift_grb = None
for c in coinc["Time [UT]"]:
    cd = datetime.strptime(c,"%H:%M:%S")
    cd = cd.replace(year = grb_date.year, month = grb_date.month, day = grb_date.day)
    if grb_time >= cd - timedelta(minutes = 2) and grb_time <= cd + timedelta(minutes = 2):
        swift_grb = coinc.loc[coinc["Time [UT]"] == c]
    else:
        print(cd)
        print(grb_time)
        print((grb_time - cd).total_seconds())

if swift_grb is not None:
    swift_grb = swift_grb.to_dict()

    poshist = os.path.join(os.environ.get("GBMDATA"), "poshist",GRB.strip("GRB")[:-3],f"glg_poshist_all_{GRB.strip('GRB')[:-3]}_v00.fit")
    trigdat = os.path.join(os.environ.get("GBMDATA"),"trigdat",str(grb_time.year),f"glg_trigdat_all_bn{GRB.strip('GRB')}_v00.fit")
    if not os.path.exists(poshist):
        download_gbm_file(date = GRB.strip("GRB")[:-3],data_type="poshist")
        print("Done downloading poshist")
    if not os.path.exists(trigdat):
        download_trigdata_file(f"bn{GRB.strip('GRB')}")
        print("Done downloading trigdat")
    interpolator = PositionInterpolator.from_poshist(poshist)
    
    if swift_grb["XRT RA (J2000)"] != "n/a":
        sgd = list(swift_grb["Date"].keys())
        swift_position = SkyCoord(ra = swift_grb["XRT RA (J2000)"][sgd[0]], dec= swift_grb["XRT Dec (J2000)"][sgd[0]],unit = (u.hourangle,u.deg))
        print(swift_position)
    else:
        print("Only BAT localization available")
    interp_trigdat = PositionInterpolator.from_trigdat(trigdat)

Total number of 1 Swift trigger(s) found
<SkyCoord (ICRS): (ra, dec) in deg
    (83.064125, 66.12383333)>


Get Incident Angle for trigger time 

In [5]:
t0 = time.Time(grb_time, format="datetime", scale="utc")
gbm_time = GBMTime(t0)
gbm = GBM(
    interpolator.quaternion(gbm_time.met),
    sc_pos=interpolator.sc_pos(gbm_time.met) * u.km,
)
sep = gbm.get_separation(swift_position)
for d in lu:
    print(d,sep[d])

n0 96.19411455288557
n1 71.88150085008424
n2 26.37443017539312
n3 115.9598343864683
n4 110.29136891470984
n5 57.206794670030526
n6 136.42387513618092
n7 160.336920161373
n8 153.5378728746127
n9 101.03889733004264
na 69.89151079895514
nb 122.96858509148946
b0 60.04709671479456
b1 119.95290328520545


In [6]:
tsbb=TimeSelectionBB(GRB,trigdat,fine=True)

16:03:34 WARNING   Minimum MC energy (5.0) is larger than minimum EBOUNDS energy                    ]8;id=864705;file:///home/tobi/sw/threeML/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=108714;file:///home/tobi/sw/threeML/threeML/utils/OGIP/response.py#137\137]8;;\
                  (3.4000000953674316)                                                                             

         WARNING   No instrument name is given. Setting to UNKNOWN                               ]8;id=499858;file:///home/tobi/sw/threeML/threeML/utils/time_series/time_series.py\time_series.py]8;;\:]8;id=793585;file:///home/tobi/sw/threeML/threeML/utils/time_series/time_series.py#140\140]8;;\

         WARNING   No mission name is given. Setting to UNKNOWN                                  ]8;id=380127;file:///home/tobi/sw/threeML/threeML/utils/time_series/time_series.py\time_series.py]8;;\:]8;id=322854;file:///home/tobi/sw/threeML/threeML/utils/time_series/time_series.py#150\150]8;;\

         WARNING   Minimum MC energy (5.0) is larger than minimum EBOUNDS energy                    ]8;id=879637;file:///home/tobi/sw/threeML/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=803105;file:///home/tobi/sw/threeML/threeML/utils/OGIP/response.py#137\137]8;;\
                  (3.4000000953674316)                                                                             

         WARNING   No instrument name is given. Setting to UNKNOWN                               ]8;id=36952;file:///home/tobi/sw/threeML/threeML/utils/time_series/time_series.py\time_series.py]8;;\:]8;id=795532;file:///home/tobi/sw/threeML/threeML/utils/time_series/time_series.py#140\140]8;;\

...

         INFO      Interval set to 0.256-9.216 for n9                                    ]8;id=88496;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py\time_series_builder.py]8;;\:]8;id=166297;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py#290\290]8;;\

         INFO      Interval set to 0.256-9.216 for na                                    ]8;id=505669;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py\time_series_builder.py]8;;\:]8;id=301985;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py#290\290]8;;\

         INFO      Interval set to 0.256-9.216 for nb                                    ]8;id=503752;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py\time_series_builder.py]8;;\:]8;id=767929;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py#290\290]8;;\

         INFO      Interval set to 0.256-9.216 for b0                                    ]8;id=998302;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py\time_series_builder.py]8;;\:]8;id=731882;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py#290\290]8;;\

         INFO      Interval set to 0.256-9.216 for b1                                    ]8;id=562546;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py\time_series_builder.py]8;;\:]8;id=80613;file:///home/tobi/sw/threeML/threeML/utils/data_builders/time_series_builder.py#290\290]8;;\

In [7]:
trigreader = tsbb.trigreader_object

In [8]:
balrog_plugin = trigreader.to_plugin(*lu)
#for i,d in enumerate(balrog_plugin):
#    balrog_plugin[i] = d.use_effective_area_correction(0.7,1.3)
datalist = DataList(*balrog_plugin)


16:04:12 INFO      Auto-probed noise models:                                                    ]8;id=5428;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=943400;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: poisson                                                       ]8;id=509028;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=490473;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#470\470]8;;\

         INFO      - background: gaussian                                                       ]8;id=984981;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=842477;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#471\471]8;;\

         INFO      Auto-probed noise models:                                                    ]8;id=717369;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=148958;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: poisson                                                       ]8;id=379360;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=334827;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#470\470]8;;\

...

         INFO      - background: gaussian                                                       ]8;id=678145;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=638557;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#471\471]8;;\

         INFO      Auto-probed noise models:                                                    ]8;id=463423;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=502001;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: poisson                                                       ]8;id=355105;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=511797;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#470\470]8;;\

         INFO      - background: gaussian                                                       ]8;id=939260;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=502765;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#471\471]8;;\

         INFO      Range c1-c6 translates to channels 1-6                                      ]8;id=521105;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=528471;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#1210\1210]8;;\

In [9]:
for d in lu:
    datalist[d].use_effective_area_correction(0.7,1.3)

         INFO      n0 is using effective area correction (between 0.7 and 1.3)                 ]8;id=657468;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=488733;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      n1 is using effective area correction (between 0.7 and 1.3)                 ]8;id=728566;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=178107;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

16:04:13 INFO      n2 is using effective area correction (between 0.7 and 1.3)                 ]8;id=136808;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=515953;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      n3 is using effective area correction (between 0.7 and 1.3)                 ]8;id=456010;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=645683;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      n4 is using effective area correction (between 0.7 and 1.3)                 ]8;id=322539;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=67440;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

...

         INFO      n9 is using effective area correction (between 0.7 and 1.3)                 ]8;id=223386;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=140470;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      na is using effective area correction (between 0.7 and 1.3)                 ]8;id=882084;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=957563;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      nb is using effective area correction (between 0.7 and 1.3)                 ]8;id=44639;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=612399;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      b0 is using effective area correction (between 0.7 and 1.3)                 ]8;id=727878;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=213722;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

         INFO      b1 is using effective area correction (between 0.7 and 1.3)                 ]8;id=134943;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=569981;file:///home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py#2192\2192]8;;\

In [10]:
cpl = Cutoff_powerlaw()
cpl.K.prior = Log_uniform_prior(lower_bound=0.0001, upper_bound=500)
cpl.xc.prior = Log_uniform_prior(lower_bound=10, upper_bound=1000)
cpl.index.set_uninformative_prior(Uniform_prior)
ps = PointSource("grb",ra = float(swift_position.ra.deg), dec = float(swift_position.dec.deg),spectral_shape=cpl)

In [11]:
ps.position.fix = True
ps.position.free = False
model = Model(ps)

In [12]:
bayes = BayesianAnalysis(model,datalist)

Freeing the position of n0 and setting priors
Freeing the position of n1 and setting priors
Freeing the position of n2 and setting priors
Freeing the position of n3 and setting priors
Freeing the position of n4 and setting priors
Freeing the position of n5 and setting priors
Freeing the position of n6 and setting priors
Freeing the position of n7 and setting priors
Freeing the position of n8 and setting priors
Freeing the position of n9 and setting priors
Freeing the position of na and setting priors
Freeing the position of nb and setting priors
Freeing the position of b0 and setting priors
Freeing the position of b1 and setting priors


In [13]:
bayes.set_sampler("multinest",share_spectrum = True)
wrap = [0] * len(model.free_parameters)  # not working properlyViel Erfolg und Spaß in den ersten Wochen!
wrap[0] = 1

bayes.sampler.setup(n_live_points=400, wrapped_params=wrap, chain_name="fit_", verbose=True)

         INFO      sampler set to multinest                                                ]8;id=174674;file:///home/tobi/sw/threeML/threeML/bayesian/bayesian_analysis.py\bayesian_analysis.py]8;;\:]8;id=462020;file:///home/tobi/sw/threeML/threeML/bayesian/bayesian_analysis.py#197\197]8;;\

In [ ]:
bayes.sample()

 *****************************************************
 MultiNest v3.12
 Copyright Farhan Feroz & Mike Hobson
 Release Nov 2019

 no. of live points =  400
 dimensionality =   19
 *****************************************************
 Starting MultiNest
 generating live points
 live points generated, starting sampling
Acceptance Rate:                        0.993377
Replacements:                                450
Total Samples:                               453
Nested Sampling ln(Z):            **************
Acceptance Rate:                        0.972763
Replacements:                                500
Total Samples:                               514
Nested Sampling ln(Z):            **************
Acceptance Rate:                        0.951557
Replacements:                                550
Total Samples:                               578
Nested Sampling ln(Z):            -691610.544260
Acceptance Rate:                        0.902256
Replacements:                              

Exception ignored on calling ctypes callback function: <function run.<locals>.loglike at 0x7f876462b1f0>
Traceback (most recent call last):
  File "/home/tobi/.venvs/localizing39/lib/python3.9/site-packages/pymultinest/run.py", line 228, in loglike
    return LogLikelihood(cube, ndim, nparams)
  File "/home/tobi/sw/threeML/threeML/bayesian/sampler_base.py", line 502, in loglike
    log_like = self._log_like(trial_values)
  File "/home/tobi/sw/threeML/threeML/bayesian/sampler_base.py", line 416, in _log_like
    log_like_values[i] = dataset.get_log_like(
  File "/home/tobi/sw/threeML/threeML/plugins/SpectrumLike.py", line 1827, in get_log_like
    loglike, _ = self._likelihood_evaluator.get_current_value(
  File "/home/tobi/sw/threeML/threeML/utils/spectrum/spectrum_likelihood.py", line 299, in get_current_value
    expected_model_counts = self._spectrum_plugin.get_model(precalc_fluxes=precalc_fluxes)
  File "/home/tobi/sw/GBM/gbm_drm_gen/gbm_drm_gen/io/balrog_like.py", line 132, in get